In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/cristian/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/cristian/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
train_entrenamiento = pd.read_csv('train_paso_3.csv')
test_entrenamiento = pd.read_csv('test_paso_3.csv')
test = pd.read_csv('trocafone_kaggle_test.csv')


In [4]:
len(train_entrenamiento)
train_entrenamiento = train_entrenamiento.drop('person', 1)

In [5]:
test_entrenamiento = test_entrenamiento.drop('person', 1)
train_entrenamiento.head()

,label,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,ad campaign hit,brand listing,checkout,conversion,generic listing,...,desconocido_modelo,Asus,LG,Lenovo,Motorola,Quantum,Samsung,Sony,iPad,iPhone
0,0,viewed product,23,checkout,1,6.0,3.0,1.0,1.0,15.0,...,43.0,0.0,2.0,0.0,0.0,0.0,11.0,1.0,0.0,11.0
1,0,ad campaign hit,1,ad campaign hit,1,1.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,viewed product,31,checkout,1,9.0,14.0,1.0,0.0,9.0,...,64.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,18.0
3,0,viewed product,24,search engine hit,1,0.0,5.0,0.0,0.0,3.0,...,13.0,0.0,0.0,6.0,5.0,0.0,10.0,3.0,0.0,0.0
4,0,viewed product,9,visited site,1,5.0,0.0,2.0,0.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0


In [6]:
#hash
#Transformo texto a numeros
train_entrenamiento['most_frequent_event'] = train_entrenamiento.apply(lambda row:  hash(row['most_frequent_event'])\
                        if not(pd.isnull(row['most_frequent_event'])) else np.nan, axis = 1)
train_entrenamiento['less_frequent_event'] = train_entrenamiento.apply(lambda row:  hash(row['less_frequent_event'])\
                        if not(pd.isnull(row['less_frequent_event'])) else np.nan, axis = 1)

In [7]:
# def normalize(df):
#     result = df.copy()
#     for feature_name in df.columns:
#         max_value = df[feature_name].max()
#         min_value = df[feature_name].min()
#         result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
#     return result

# train_entrenamiento=normalize(train_entrenamiento)
# train_entrenamiento.head()

In [8]:
string_columns=train_entrenamiento.columns
string_columns

Index(['label', 'most_frequent_event', 'count_most_frequent_event',
       'less_frequent_event', 'count_less_frequent_event', 'ad campaign hit',
       'brand listing', 'checkout', 'conversion', 'generic listing', 'lead',
       'search engine hit', 'searched products', 'staticpage',
       'viewed product', 'visited site', 'mean_per_event', '128GB', '16GB',
       '256GB', '32GB', '4GB', '512MB', '64GB', '8GB', 'Bom',
       'Bom - Sem Touch ID', 'Excelente', 'Muito Bom', 'Novo',
       'desconocido_modelo', 'Asus', 'LG', 'Lenovo', 'Motorola', 'Quantum',
       'Samsung', 'Sony', 'iPad', 'iPhone'],
      dtype='object')

In [9]:
train = train_entrenamiento.drop('label', 1)
train.head()

,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,...,desconocido_modelo,Asus,LG,Lenovo,Motorola,Quantum,Samsung,Sony,iPad,iPhone
0,-8045053837070434029,23,-4369631690812115373,1,6.0,3.0,1.0,1.0,15.0,0.0,...,43.0,0.0,2.0,0.0,0.0,0.0,11.0,1.0,0.0,11.0
1,7673245120682117780,1,7673245120682117780,1,1.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-8045053837070434029,31,-4369631690812115373,1,9.0,14.0,1.0,0.0,9.0,0.0,...,64.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,18.0
3,-8045053837070434029,24,-7270690298717534081,1,0.0,5.0,0.0,0.0,3.0,0.0,...,13.0,0.0,0.0,6.0,5.0,0.0,10.0,3.0,0.0,0.0
4,-8045053837070434029,9,7172899938073839000,1,5.0,0.0,2.0,0.0,0.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0


In [10]:
trainlabel = train_entrenamiento['label']
trainlabel.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [11]:
n = 500
knn = KNeighborsRegressor(n_neighbors = n)

In [30]:
parameters_for_testing = {
    'n_neighbors':[110,150,200,250],
}

In [31]:
gsearch1 = GridSearchCV(estimator = knn, param_grid = parameters_for_testing,\
                       n_jobs=6,iid=False, verbose=10,scoring='roc_auc')
gsearch1.fit(train,trainlabel)
print (gsearch1.grid_scores_)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] n_neighbors=110 .................................................
[CV] n_neighbors=110 .................................................
[CV] n_neighbors=110 .................................................
[CV] n_neighbors=150 .................................................
[CV] n_neighbors=150 .................................................
[CV] n_neighbors=150 .................................................
[CV] ........................ n_neighbors=110, score=0.733171 -   3.9s
[CV] n_neighbors=200 .................................................
[CV] ........................ n_neighbors=110, score=0.737038 -   3.9s
[CV] n_neighbors=200 .................................................


[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    3.9s


[CV] ........................ n_neighbors=110, score=0.722322 -   4.5s
[CV] n_neighbors=200 .................................................


[Parallel(n_jobs=6)]: Done   3 out of  12 | elapsed:    4.5s remaining:   13.6s


[CV] ........................ n_neighbors=150, score=0.730589 -   4.7s
[CV] ........................ n_neighbors=150, score=0.735156 -   4.7s
[CV] n_neighbors=250 .................................................
[CV] n_neighbors=250 .................................................


...

[CV] ........................ n_neighbors=200, score=0.728822 -   4.4s
[CV] ........................ n_neighbors=200, score=0.730981 -   4.9s


[Parallel(n_jobs=6)]: Done   9 out of  12 | elapsed:    9.0s remaining:    3.0s


[CV] ........................ n_neighbors=250, score=0.731261 -   4.6s
[CV] ........................ n_neighbors=250, score=0.728367 -   4.8s
[CV] ........................ n_neighbors=250, score=0.729728 -   4.7s


[Parallel(n_jobs=6)]: Done  12 out of  12 | elapsed:    9.8s finished


[mean: 0.73084, std: 0.00623, params: {'n_neighbors': 110}, mean: 0.73325, std: 0.00194, params: {'n_neighbors': 150}, mean: 0.72983, std: 0.00089, params: {'n_neighbors': 200}, mean: 0.72979, std: 0.00118, params: {'n_neighbors': 250}]
best params
{'n_neighbors': 150}
best score
0.7332465780429039


In [22]:
n = 600
knn = KNeighborsRegressor(n_neighbors = n)
knn.fit(train, trainlabel)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=600, p=2,
          weights='uniform')

In [23]:
#hash
#Transformo texto a numeros
test_entrenamiento['most_frequent_event'] = test_entrenamiento.apply(lambda row:  hash(row['most_frequent_event'])\
                        if not(pd.isnull(row['most_frequent_event'])) else np.nan, axis = 1)
test_entrenamiento['less_frequent_event'] = test_entrenamiento.apply(lambda row:  hash(row['less_frequent_event'])\
                        if not(pd.isnull(row['less_frequent_event'])) else np.nan, axis = 1)

In [24]:
# def normalize(df):
#     result = df.copy()
#     for feature_name in df.columns:
#         max_value = df[feature_name].max()
#         min_value = df[feature_name].min()
#         result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
#     return result

# test_entrenamiento=normalize(test_entrenamiento)
# test_entrenamiento.head()

In [25]:
testData_sinId = test_entrenamiento

In [26]:
prediction = knn.predict(testData_sinId)

In [27]:
ids = test['person'].values
dataKaggle = pd.DataFrame(ids ,columns={'person'})
dataKaggle.head(5)

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [28]:
#Aca agregamos una columna que seria se postulop
dataKaggle['label'] = prediction
dataKaggle.head()

,person,label
0,4886f805,0.028333
1,0297fc1e,0.046667
2,2d681dd8,0.041667
3,cccea85e,0.028333
4,4c8a8b93,0.028333


In [29]:
dataKaggle.to_csv("predictionKnn.csv", index = False)